In [1]:
import pandas as pd
import datetime
from configparser import ConfigParser
import json, requests, datetime
from pysimplicate import Simplicate

# Pandas
pd.options.display.float_format = '{:,.1f}'.format
pd.set_option('display.max_columns', 500)
PANDAS_FILE = '../simplicate_cache/hours.pd'
df = pd.read_pickle(PANDAS_FILE)

df = df.query( 'type=="normal"')
df['turnover'] = df.apply(lambda a: (a['hours']+a['corrections']) * (a['tariff'] if a['tariff'] > 0 else a['service_tariff']), axis=1)
df['turnover'] = df.apply(lambda a: a['turnover']/2 if a['project_number'] == 'TOR-3' else a['turnover'], axis=1)
df['week'] = df.apply(lambda a: datetime.datetime.strptime(a['day'],'%Y-%m-%d').isocalendar()[1], axis=1)
df['month'] = df.apply(lambda a: datetime.datetime.strptime(a['day'],'%Y-%m-%d').month, axis=1)

# Simplicate
ini = ConfigParser()
ini.read('../credentials.ini')

subdomain = ini['simplicate']['subdomain']
api_key = ini['simplicate']['api_key']
api_secret = ini['simplicate']['api_secret']

sim = Simplicate(subdomain, api_key, api_secret )

In [2]:
df[-3:]

,employee,organization,project_id,project_name,project_number,service,type,service_tariff,label,billable,tariff,hours,day,status,corrections,turnover,week,month
7079,Hans-Peter Harmsen,Oberon,project:21ea648f0c0c31fcfeaad60b7a7437df,Internal,OBE-1,MT,normal,0.0,Internal,False,0.0,3.2,2021-03-10,to_forward,0.0,0.0,10,3
7080,Hans-Peter Harmsen,Oberon,project:21ea648f0c0c31fcfeaad60b7a7437df,Internal,OBE-1,Office,normal,0.0,Internal,False,0.0,0.2,2021-03-10,to_forward,0.0,0.0,10,3
7081,Hans-Peter Harmsen,Oberon,project:21ea648f0c0c31fcfeaad60b7a7437df,Internal,OBE-1,HRM,normal,0.0,Internal,False,0.0,0.5,2021-03-10,to_forward,0.0,0.0,10,3


## Turnover

In [3]:
def turnover( project, month=None, from_date=None ):
    query = f'project_number=="{project}"'
    if month: 
        query += f' and month=={month}'
    if from_date:
        query += f' and day>="{from_date}"'
    data = df.query(query)
    return data['turnover'].sum()

turnover( 'BAM-1', from_date='2021-02-01')

33268.575

## Invoices

In [4]:
def invoiced(project, month=None ):
    filter = {'project_number':project}
    if month:
        filter['from_date'] = f'2021-0{month}-01'
        filter['until_date'] = f'2021-0{month+1}-01'
    invoices = sim.invoice( filter )
    tot = 0
    for invoice in invoices:
        #print( invoice.get('invoice_number','????'), invoice['total_excluding_vat'], invoice['status'])
        tot += invoice['total_excluding_vat']
    return tot

def last_invoice_date( project ):
    invoices = sim.invoice( {'project_number':project} )
    invoices = sorted( invoices, key=lambda i: i['date'])
    if invoices:
        return datetime.datetime.strptime( invoices[-1]['date'], '%Y-%m-%d').date()
    
#print( invoiced( 'BAM-1'))
print( last_invoice_date( 'SLIM-16' ))

2020-11-30


## All projects, certain month

In [5]:
def active_projects():
    projects = [{'project': project.get('project_number',''),
                 'spent' : project['budget']['hours'].get('value_spent', 0),
                 'invoiced' : project['budget']['total']['value_invoiced']
                }
                for project in sim.project( {'active':True} )]
    return projects
active_projects()[:3]

[{'project': 'OBE-1', 'spent': 0, 'invoiced': 0},
 {'project': 'TEX-1', 'spent': 72772.5, 'invoiced': 0},
 {'project': 'TOR-3', 'spent': 115776.25, 'invoiced': 6412.5}]

## Onderhanden werk

In [6]:
pd.set_option('display.max_row', 150)

# project['budget']['total']['value_invoiced']
def corrections(project):
    h = df.query( 'project_number=="{project}"' )
    return h['corrections'].sum()
    
def onderhanden():
    return pd.DataFrame( [{'project':project['project'], 
                           'spent':project['spent'],
                           'corr': corrections( project),
                           'inv': project['invoiced'],
                           'OH':project['spent'] + corrections( project) - project['invoiced']} 
                          for project in active_projects()] ).sort_values( by=['OH'])
oh = onderhanden()
oh.drop( oh[(oh.project=='TOR-3')].index, inplace=True)
oh

,project,spent,corr,inv,OH
50,TEX-2,"2,475.0",0.0,"10,047.5","-7,572.5"
3,SM2021,"37,336.2",0.0,"44,370.0","-7,033.8"
42,BITF-1,"4,972.5",0.0,"10,880.0","-5,907.5"
44,ONC-2,"5,185.0",0.0,"10,200.0","-5,015.0"
4,HAVA-2,"16,301.2",0.0,"21,225.0","-4,923.8"
64,AME-1,137.5,0.0,"4,457.5","-4,320.0"
76,GREE-6,"11,156.2",0.0,"15,000.0","-3,843.8"
30,VOLK-1,"3,440.2",0.0,"7,120.5","-3,680.2"
67,LEAN-2,0.0,0.0,"3,600.0","-3,600.0"
69,TER-1,495.0,0.0,"3,380.0","-2,885.0"


In [7]:
oh['OH'].sum()

275665.412467

# Correcties

In [8]:
df[-3:]

,employee,organization,project_id,project_name,project_number,service,type,service_tariff,label,billable,tariff,hours,day,status,corrections,turnover,week,month
7079,Hans-Peter Harmsen,Oberon,project:21ea648f0c0c31fcfeaad60b7a7437df,Internal,OBE-1,MT,normal,0.0,Internal,False,0.0,3.2,2021-03-10,to_forward,0.0,0.0,10,3
7080,Hans-Peter Harmsen,Oberon,project:21ea648f0c0c31fcfeaad60b7a7437df,Internal,OBE-1,Office,normal,0.0,Internal,False,0.0,0.2,2021-03-10,to_forward,0.0,0.0,10,3
7081,Hans-Peter Harmsen,Oberon,project:21ea648f0c0c31fcfeaad60b7a7437df,Internal,OBE-1,HRM,normal,0.0,Internal,False,0.0,0.5,2021-03-10,to_forward,0.0,0.0,10,3


In [31]:
import datetime
one_week_ago = (datetime.datetime.today() + datetime.timedelta(weeks=-1)).strftime(DATE_FORMAT)
five_weeks_ago = (datetime.datetime.today() + datetime.timedelta(weeks=-5)).strftime(DATE_FORMAT)
data = df.query(f'(tariff>0 or service_tariff>0) and day>="{one_week_ago}" and day<"{five_weeks_ago}"')
data

,employee,organization,project_id,project_name,project_number,service,type,service_tariff,label,billable,tariff,hours,day,status,corrections,turnover,week,month
6770,Rogier Voogt,T-Mobile Netherlands B.V.,project:921a9486c1cbd8e6feaad60b7a7437df,BEN App,BEN-1,Development Sprints Q1,normal,110.0,App development,True,77.5,8.0,2021-03-05,projectmanager_approved,0.0,620.0,9,3
6772,Richard van Willegen,Accell IT,project:ba493253e99f604dfeaad60b7a7437df,Headless 2021,ACC-1,Development Sprints Maart,normal,95.0,Front-end Development,True,135.0,7.5,2021-03-05,projectmanager_approved,0.0,"1,012.5",9,3
6773,Jordi Hendrix,TOR groep,project:5fdd3f799c981c6dfeaad60b7a7437df,TOR 3.0,TOR-3,TOR 3.0 Fase 2 Schiermonnikoog 2021,normal,47.5,Back-end Development,True,47.5,7.5,2021-03-05,projectmanager_approved,0.0,178.1,9,3
6782,Eva Korteweg,VVV Texel,project:2e7d1588e87a48a8feaad60b7a7437df,Website,TEX-1,Development Sprints 2021,normal,110.0,Project Management,True,80.0,4.5,2021-03-05,forwarded,0.0,360.0,9,3
6783,Eva Korteweg,TOR groep,project:5fdd3f799c981c6dfeaad60b7a7437df,TOR 3.0,TOR-3,Development Sprints Q1,normal,55.0,Project Management,False,38.8,2.0,2021-03-05,projectmanager_approved,0.0,38.8,9,3
6785,Richard de Boer,Sprout Money BV,project:6c7adeccdadb36f4feaad60b7a7437df,BitFirst Exchange,BITF-1,Setting up Binance portal (strippenkaart 1),normal,110.0,Maintenance & Support,False,85.0,1.5,2021-03-05,projectmanager_approved,0.0,127.5,9,3
6786,Richard de Boer,VVV Texel,project:2e7d1588e87a48a8feaad60b7a7437df,Website,TEX-1,Strippenkaart,normal,110.0,Maintenance & Support,False,110.0,1.0,2021-03-05,forwarded,0.0,110.0,9,3
6791,Richard de Boer,TOR groep,project:5fdd3f799c981c6dfeaad60b7a7437df,TOR 3.0,TOR-3,Development Sprints Q1,normal,55.0,Project Management,False,55.0,4.0,2021-03-05,projectmanager_approved,0.0,110.0,9,3
6792,Jari Zwarts,VVV Texel,project:2e7d1588e87a48a8feaad60b7a7437df,Website,TEX-1,Development Sprints 2021,normal,95.0,Front-end Development,True,105.0,8.0,2021-03-05,forwarded,0.0,840.0,9,3
6794,Stefan Roovers,Sprout Money BV,project:8a11f2a1ccca7569feaad60b7a7437df,Value Jagers Agile,VJ2021,Value Jagers - Maart 2021,normal,110.0,Project Management,True,85.0,0.2,2021-03-05,projectmanager_approved,0.0,21.2,9,3


In [33]:
df['day'].min()

'2021-01-01'

In [36]:
one_week_ago = (datetime.datetime.today() + datetime.timedelta(weeks=-1)).strftime(DATE_FORMAT)
five_weeks_ago = (datetime.datetime.today() + datetime.timedelta(weeks=-5)).strftime(DATE_FORMAT)
query = f'(tariff>0 or service_tariff>0) and day>="{five_weeks_ago}" and day<"{one_week_ago}"'
print( query )
data = df.query(query)
print( data['day'].min() )
print( data['day'].max() )
print( data['hours'].sum() )
percentage_corrected = 100 * -data['corrections'].sum() / data['hours'].sum()
percentage_corrected

(tariff>0 or service_tariff>0) and day>="2021-02-05" and day<"2021-03-05"
2021-02-05
2021-03-04
3802.583333333333


4.998794679055028

In [10]:
DATE_FORMAT = '%Y-%m-%d'
lastmonth = (datetime.datetime.today() + datetime.timedelta(days=-30)).strftime(DATE_FORMAT)
a = df.query(f'corrections < 0 and day>="{lastmonth}"').groupby(['organization','project_name']).agg({'hours':'sum','corrections':'sum', 'turnover':'sum'}).sort_values('corrections').query('corrections < -10')
a.reset_index()

,organization,project_name,hours,corrections,turnover
0,ThiemeMeulenhoff B.V.,Examenbundel onderhoud en doorontwikkeling,40.0,-40.0,0.0
1,Oncode,Oncode Community aanpassingen,38.8,-38.8,0.0
2,ThiemeMeulenhoff B.V.,Examenbundel Design Sprint,24.0,-24.0,0.0


## Correcties op een specifiek project

In [11]:
easy = df.query('project_number == "EASY-1"')
easy

,employee,organization,project_id,project_name,project_number,service,type,service_tariff,label,billable,tariff,hours,day,status,corrections,turnover,week,month
566,Sebastian Schipper,EasyBroker,project:9b63e1bed41e11eefeaad60b7a7437df,Redesign website,EASY-1,Redesign website,normal,95.0,Product & UX design,False,85.0,1.0,2021-01-07,projectmanager_approved,0.0,85.0,1,1
695,Sebastian Schipper,EasyBroker,project:9b63e1bed41e11eefeaad60b7a7437df,Redesign website,EASY-1,Redesign website,normal,95.0,UX/UI Design,False,85.0,2.0,2021-01-08,projectmanager_approved,0.0,170.0,1,1
993,Sebastian Schipper,EasyBroker,project:9b63e1bed41e11eefeaad60b7a7437df,Redesign website,EASY-1,Redesign website,normal,95.0,Product & UX design,False,85.0,2.0,2021-01-12,projectmanager_approved,0.0,170.0,2,1
1039,Sebastian Schipper,EasyBroker,project:9b63e1bed41e11eefeaad60b7a7437df,Redesign website,EASY-1,Redesign website,normal,95.0,Product & UX design,False,85.0,3.0,2021-01-12,projectmanager_approved,0.0,255.0,2,1
1152,Sebastian Schipper,EasyBroker,project:9b63e1bed41e11eefeaad60b7a7437df,Redesign website,EASY-1,Redesign website,normal,95.0,Product & UX design,False,85.0,2.0,2021-01-13,projectmanager_approved,0.0,170.0,2,1
1184,Sebastian Schipper,EasyBroker,project:9b63e1bed41e11eefeaad60b7a7437df,Redesign website,EASY-1,Redesign website,normal,95.0,Product & UX design,False,85.0,3.0,2021-01-13,projectmanager_approved,0.0,255.0,2,1
1241,Sebastian Schipper,EasyBroker,project:9b63e1bed41e11eefeaad60b7a7437df,Redesign website,EASY-1,Redesign website,normal,95.0,Product & UX design,False,85.0,2.5,2021-01-14,projectmanager_approved,0.0,212.5,2,1
1329,Sebastian Schipper,EasyBroker,project:9b63e1bed41e11eefeaad60b7a7437df,Redesign website,EASY-1,Redesign website,normal,110.0,Project Management,False,85.0,2.5,2021-01-14,projectmanager_approved,0.0,212.5,2,1
1340,Joost Cornelissen,EasyBroker,project:9b63e1bed41e11eefeaad60b7a7437df,Redesign website,EASY-1,Redesign website,normal,125.0,Creative Direction,False,85.0,2.0,2021-01-14,projectmanager_approved,0.0,170.0,2,1
1741,Patricia Snel,EasyBroker,project:9b63e1bed41e11eefeaad60b7a7437df,Redesign website,EASY-1,Redesign website,normal,95.0,UX/UI Design,False,85.0,4.0,2021-01-19,projectmanager_approved,0.0,340.0,3,1


In [12]:
easy.groupby(['organization','project_name','project_id']).agg({'hours':'sum','corrections':'sum', 'turnover':'sum'}).query('corrections < -10')

,,,hours,corrections,turnover
organization,project_name,project_id,,,
EasyBroker,Redesign website,project:9b63e1bed41e11eefeaad60b7a7437df,126.5,-11.0,"9,817.5"


In [13]:
df.query(f'corrections < 0 and project_number == "EASY-1"')\
        .groupby(['organization', 'project_name', 'project_id'])\
        .agg({'hours': 'sum', 'corrections': 'sum'})\
        .sort_values('corrections')\
        .reset_index()

,organization,project_name,project_id,hours,corrections
0,EasyBroker,Redesign website,project:9b63e1bed41e11eefeaad60b7a7437df,11.0,-11.0


In [14]:
df.query('corrections < 0').query('project_number == "EASY-1"').groupby(['organization','project_name','project_id']).agg({'hours':'sum','corrections':'sum', 'turnover':'sum'}).query('corrections < -10')

,,,hours,corrections,turnover
organization,project_name,project_id,,,
EasyBroker,Redesign website,project:9b63e1bed41e11eefeaad60b7a7437df,11.0,-11.0,0.0


In [15]:
df.query('corrections < 0 and project_number == "EASY-1"')

,employee,organization,project_id,project_name,project_number,service,type,service_tariff,label,billable,tariff,hours,day,status,corrections,turnover,week,month
1746,Sho Stegmeijer,EasyBroker,project:9b63e1bed41e11eefeaad60b7a7437df,Redesign website,EASY-1,Redesign website,normal,95.0,UX/UI Design,False,85.0,3.0,2021-01-19,projectmanager_approved,-3.0,0.0,3,1
2546,Sho Stegmeijer,EasyBroker,project:9b63e1bed41e11eefeaad60b7a7437df,Redesign website,EASY-1,Redesign website,normal,95.0,Product & UX design,False,85.0,3.0,2021-01-26,projectmanager_approved,-3.0,0.0,4,1
2584,Sho Stegmeijer,EasyBroker,project:9b63e1bed41e11eefeaad60b7a7437df,Redesign website,EASY-1,Redesign website,normal,95.0,Product & UX design,False,85.0,5.0,2021-01-26,projectmanager_approved,-5.0,0.0,4,1
